In [1]:
# 0:clicks 1:carts 2:orders
type_weight = {0:0.5,
               1:9,
               2:0.5}
type_weight_multipliers = type_weight

# Use top X for clicks, carts and orders Top何位までを使うか
clicks_th = 40 
carts_th  = 100 
orders_th = 100

VER = 1
DIR = f'/home/anhphantq/vdb/covisit_ver_{VER}'

In [2]:
import os 
try:
    os.mkdir(DIR)
    
    os.mkdir(DIR + '/splitted')
    os.mkdir(DIR + '/lb')
except OSError as error:
    print(error)    

[Errno 17] File exists: '/home/anhphantq/vdb/covist_ver_1'


In [24]:
import pandas as pd

# COMPUTE IN PARTS FOR MEMORY MANGEMENT

train_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/train.parquet')
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

CPU times: user 5.13 s, sys: 4.9 s, total: 10 s
Wall time: 4.05 s


(1661119199, 1659304800)

In [3]:
import pandas as pd
train_df = pd.read_parquet('/home/anhphantq/otto/data/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
old_shape = train_df.shape[0]
old_num_aid = len(np.unique(train_df.aid))
train_df = train_df.drop(old_index.flatten())
num_aid = len(np.unique(train_df.aid))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
train_df.shape[0], old_shape,  old_shape - train_df.shape[0], num_aid, old_num_aid, old_num_aid - num_aid, MAX_TS, MIN_TS

(163555218,
 216716096,
 53160878,
 1839558,
 1855603,
 16045,
 1661723999,
 1659909600)

In [4]:
def create_cart_order(args):
    df, batch_id = args[0], args[1]
    df = df.sort_values(['session','ts'],ascending=[True,False])

# USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]


    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y', 'type_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 100
    df = df.loc[df.n < carts_th].drop('n', axis = 1)
    return df, batch_id


def create_buy2buy(args):
    df, batch_id = args[0], args[1]
    
    
    df = df.loc[df['type'].isin([1,2])]
    df = df.sort_values(['session','ts'],ascending=[True,False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]


    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y', 'type_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 100
    df = df.loc[df.n < orders_th].drop('n', axis = 1)
    return df, batch_id

def create_click(args):
    df, batch_id = args[0], args[1]
    
    
    df = df.loc[df['type'].isin([1,2])]
    df = df.sort_values(['session','ts'],ascending=[True,False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y) ]


    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = 1 + 3*(df.ts_x - MIN_TS)/(MAX_TS-MIN_TS)
    
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 100
    df = df.loc[df.n < clicks_th].drop('n', axis = 1)
    return df, batch_id



In [5]:
from tqdm import tqdm
import numpy as np

step = train_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

batches = [train_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]
batch_names = [i for i in range(len(batches))]




In [28]:
#splitted
import multiprocessing
print('create top clicks')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_cart_order,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/splitted/top_carts_orders_frag_{batch_id}', index = True)
  
print('create top buy2buy')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_buy2buy,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/splitted/top_buy2buy_frag_{batch_id}', index = True)

print('create top clicks')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_click,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/splitted/top_click_frag_{batch_id}', index = True)


create top clicks


100%|██████████| 64/64 [01:28<00:00,  1.38s/it]


create top buy2buy


100%|██████████| 64/64 [00:11<00:00,  5.45it/s]


create top clicks


100%|██████████| 64/64 [00:12<00:00,  5.27it/s]


In [6]:
#lb
import multiprocessing
print('create top clicks')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_cart_order,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/lb/top_carts_orders_frag_{batch_id}', index = True)
  
print('create top buy2buy')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_buy2buy,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/lb/top_buy2buy_frag_{batch_id}', index = True)

print('create top clicks')
with multiprocessing.Pool(64) as pool:
  for rs, batch_id in tqdm(pool.imap_unordered(create_click,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
    rs.to_parquet(DIR + f'/lb/top_click_frag_{batch_id}', index = True)


create top clicks


100%|██████████| 64/64 [01:28<00:00,  1.38s/it]


create top buy2buy


100%|██████████| 64/64 [00:15<00:00,  4.07it/s]


create top clicks


100%|██████████| 64/64 [00:15<00:00,  4.04it/s]
